In [1]:
import pandas as pd
import time
import nltk

In [2]:
class nlp:
    #need to download from nltk - 'averaged_perceptron_tagger','wordnet','stopwords'
    re = __import__('re')
    pd = __import__('pandas')
    nltk = __import__('nltk')
    
    def __init__(self):
        print('nlp V.0.1 \nImported pandas,re,nltk packages')
        
    def clean_text_for_topic_modelling(self,text_column):
        text_column=text_column.str.lower()
        text_column = [self.re.sub(r'https?:\/\/*[^ ]*', '', x) for x in text_column]
        text_column = [self.re.sub(r'[.,;/]',' ', x) for x in text_column]
        text_column = [self.re.sub(r'\((cont)\)','', x) for x in text_column]
        text_column = [self.re.sub(r'[^A-Za-z0-9$% ]','', x) for x in text_column]
        text_column = [x.split() for x in text_column]
        temp_corpus=[]
        for tweet in text_column:
            new_list=[w for w in tweet if len(w)>2]
            temp_corpus.append(new_list)
        text_column=self.pd.Series(temp_corpus)
        return text_column
    
    def remove_stopwords_from_corpus(self,text_column,extra_stopwords_list=[]):
        from nltk.corpus import stopwords
        stops = set(stopwords.words('english')).union(extra_stopwords_list)
        text_column=text_column.apply(lambda x: [w for w in x if w not in stops])
        return text_column
    
    def stemm_corpus(self,text_column,stemmer='porter'):
        if stemmer=='porter':
            from nltk.stem import PorterStemmer
            porter = PorterStemmer()
            text_column=text_column.apply(lambda x:[porter.stem(w) for w in x])
        elif stemmer=='lancaster':
            from nltk.stem import LancasterStemmer
            lancaster=LancasterStemmer()
            text_column=text_column.apply(lambda x:[lancaster.stem(w) for w in x])
        else:
            from nltk.stem import SnowballStemmer
            snowball=SnowballStemmer("english")
            text_column=text_column.apply(lambda x:[snowball.stem(w) for w in x])
        return text_column
    
    def lemmatize_corpus(self,text_column):
        from nltk.stem import WordNetLemmatizer
        from nltk.corpus import wordnet
        
        def get_word_pos(word):
            tag = self.nltk.pos_tag(word)[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
            return tag_dict.get(tag, wordnet.NOUN)
        
        wordnet_lemmatizer = WordNetLemmatizer()
        text_column=text_column.apply(lambda x:[wordnet_lemmatizer.lemmatize(w, pos=get_word_pos(w)) for w in x])
        return text_column
    
    def create_ngrams(self,text_column,replace=False,num_grams=2):
        from nltk import ngrams
        if replace:
            grams_list=[]
            for tweet in text_column:
                grams_list.append([' '.join(ngram) for ngram in ngrams(tweet,num_grams)])
            return self.pd.Series(grams_list)
        else:
            for i,tweet in text_column.iteritems():
                copy_tweet=tweet.copy()
                for ngram in ngrams(tweet,2):
                    copy_tweet.append(' '.join(ngram))
                text_column.at[i]=copy_tweet
            return text_column
        
    def gensim_dic(self,text_column,filter_extremes=True,no_below=3,no_above=0.99):
        from gensim.corpora import Dictionary
        dictionary=Dictionary(corpus)
        if filter_extremes:
            dictionary.filter_extremes(no_below=no_below, no_above=no_above)
        return dictionary
    
    def bow_corpus(self,dictionary,text_column):
        return [dictionary.doc2bow(doc) for doc in text_column]
    
    def vec_to_tfidf(self,bow_corpus):
        from gensim import models
        tfidf = models.TfidfModel(bow_corpus)
        return tfidf[bow_corpus]
    
    def lda_model(self,tfidf_corp,dic,num_topics=20,passes=30,alpha=0.001,eta='auto'):
        from gensim import models
        return models.ldamodel.LdaModel(corpus=tfidf_corp, num_topics=num_topics, id2word=dic, passes=passes, alpha=alpha, 
                                             eta=eta,random_state=13)
    
    def get_coherence(self,lda_model,corpus,dic):
        from gensim.models import CoherenceModel
        coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=dic, coherence='c_v')
        return coherence_model_lda.get_coherence()
    
    def is_tweet_intopic(self,text_column,words_list):
        
        def is_tweets_words_inlist(tweet,words_list):
            for word in tweet:
                if word in words_list:
                    return True
            return False
        
        results=[]
        for tweet in text_column:
            results.append(is_tweets_words_inlist(tweet,words_list))
        return self.pd.Series(results)
    
    def assign_lda_topic(self,lda_model,bow_corpus):
        topics=[]
        data=lda_model.get_document_topics(bow_corpus)
        for tweet in data:  
            df = self.pd.DataFrame(tweet, columns=['topic_num', 'probability'])
            topics.append(df.loc[df['probability'].idxmax(), 'topic_num'])
        return (self.pd.Series(topics))
    
    def get_sentiment(self,text_column):
        from textblob import TextBlob
        return text_column.apply(lambda x: TextBlob(' '.join(x)).sentiment.polarity)

In [3]:
df=pd.read_csv('trump_tweets_reducted.csv',parse_dates=['created_at_utc'])
df.reset_index(inplace=True,drop=True)

In [4]:
nlpob=nlp()

nlp V.0.1 
Imported pandas,re,nltk packages


In [5]:
corpus=nlpob.clean_text_for_topic_modelling(df['text'])

In [6]:
corpus=nlpob.remove_stopwords_from_corpus(corpus,['realdonaldtrump','amp','president','android','iphone'])

In [7]:
corpus_without_stemorlem=corpus.copy()

In [8]:
corpus=nlpob.stemm_corpus(corpus)

In [9]:
corpus=nlpob.lemmatize_corpus(corpus)

In [10]:
corpus=nlpob.create_ngrams(corpus)

In [11]:
topic_words=['deal','deals','china','chinese','nafta','market','markets','stock','stocks','trade','trades','trading','tax',
             'taxes','taxation','rate','rates','unemployment','jobs','manafacture','manafacturers','dollar','dollars','$',
             'billion','billions','consumer','consumers','reserve','gdp','growth','bank','banks','debt','factories',
             'revenue','spending','deficit','economy','economies','economist','economic','economists','money',
             'tariff','tariffs','companies','inflation','refinance','finance','financial','currency','depreciating',
             'depreceate','fed','business','businesses','bankruptcy','price','prices','products','opec','investment',
             'savings','cents','cent','corporate','corporates','deficits','dow','nasdaq','labor','purchase','500',
             'industry','industries']

In [12]:
df['financial_topic']=nlpob.is_tweet_intopic(corpus_without_stemorlem,topic_words)

In [13]:
dic=nlpob.gensim_dic(corpus)
bow_corp=nlpob.bow_corpus(dic,corpus)
tfidf_corp=nlpob.vec_to_tfidf(bow_corp)
lda_model = nlpob.lda_model(tfidf_corp,dic)
nlpob.get_coherence(lda_model,corpus,dic)

0.43284546532459034

In [14]:
df['lda_topic'] = nlpob.assign_lda_topic(lda_model,bow_corp)

In [15]:
df['sentiment'] = nlpob.get_sentiment(corpus_without_stemorlem)

In [16]:
df.to_csv('trump_tweets_reducted_after_nlp.csv')

In [20]:
df[df['financial_topic']==True]['text']

2        Donald Trump reads Top Ten Financial Tips on L...
22       “If you don't have problems you're pretending ...
31       Watch a powerful and frank interview with Dona...
37       Browse Donald Trump's Summer Reading List for ...
48       Watch video of Ivanka Trump sharing business a...
                               ...                        
25663    ....absolutely no pressure. I don’t know of an...
25668    Chuck Schumer sat for years during the Obama A...
25695    ....energy independence manufacturing resilien...
25696    ....of the most successful presidencies in his...
25698    ....energy independence manufacturing resilien...
Name: text, Length: 3688, dtype: object

In [17]:
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} Word: {}'.format(idx, topic))

import pyLDAvis
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corp, dic)
vis

C:\Users\natan\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.191914 -0.036964       1        1  13.069150
4     -0.177062 -0.009302       2        1  11.726679
17    -0.160400 -0.024559       3        1   9.058296
7     -0.120435 -0.006562       4        1   7.816061
19    -0.114766  0.019759       5        1   7.213919
9     -0.044586  0.022344       6        1   5.895144
10    -0.026698  0.084019       7        1   5.703498
11    -0.051005 -0.090170       8        1   4.914306
18    -0.012586  0.015148       9        1   4.749685
3      0.065382  0.199025      10        1   4.451365
6      0.001736 -0.012336      11        1   3.599144
1      0.040165  0.059550      12        1   3.278890
5      0.006993 -0.070421      13        1   3.218255
0      0.071717  0.078944      14        1   3.127552
8      0.030421  0.009745      15        1   2.798399
15     0.103327 -0.002577      16        1   2.375238
16     0.115641 -0.066670      17        1   2.006399
12     0.144616 -0.092848      18        1   1.876363
14     0.171086 -0.023009      19        1   1.604486
2      0.148366 -0.053117      20        1   1.517172, topic_info=     Category         Freq           Term        Total  loglift  logprob
175   Default  2821.000000          thank  2821.000000  30.0000  30.0000
5233  Default   668.000000        hillari   668.000000  29.0000  29.0000
2506  Default   592.000000   make america   592.000000  28.0000  28.0000
1528  Default   587.000000  america great   587.000000  27.0000  27.0000
1767  Default   785.000000         border   785.000000  26.0000  26.0000
...       ...          ...            ...          ...      ...      ...
1353  Topic20    39.738972         enough   115.218765   3.1238  -4.9308
2188  Topic20    27.119923          treat   148.514175   2.4879  -5.3129
2215  Topic20    23.869654          death    54.637966   3.3602  -5.4406
887   Topic20    23.279680        septemb    45.914757   3.5091  -5.4656
1788  Topic20    25.502815            sad   225.888275   2.0071  -5.3744

[1063 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1543      1  0.944015   100%
1543      5  0.031467   100%
3906     11  0.898637    1st
5017      9  0.096068   2016
5017     11  0.806385   2016
...     ...       ...    ...
3311     17  0.183997  young
1098      8  0.941876   youv
2975      3  0.564253   zero
2975     18  0.376168   zero
2975     19  0.036170   zero

[2634 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 5, 18, 8, 20, 10, 11, 12, 19, 4, 7, 2, 6, 1, 9, 16, 17, 13, 15, 3])

In [ ]:
## hdpmodel
# from gensim.corpora import Dictionary
# dct = Dictionary(corpus)
# vector_corpus=[dct.doc2bow(tweet) for tweet in corpus]
# from gensim.models import HdpModel
# hdp = HdpModel(vector_corpus, dct,alpha=0.001)
# topic_info = hdp.print_topics(num_topics=10, num_words=20)
# topic_info

In [ ]:
# compare=pd.DataFrame({'corpus':corpus,'real':df['text']})
# for i,row in compare[17000:].iterrows():
#     print(row['corpus'])
#     print('\n')
#     print(row['real'])
#     time.sleep(2)